In [1]:
import os
import langchain
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage 

os.environ['OPENAI_API_KEY'] = ''


os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = ''

chat_model = ChatOpenAI(temperature=0.0,openai_api_key=os.getenv("OPENAI_API_KEY"))
chat_model
chat_model.predict("hi!")

text = "I am looking to book a direct flight from New York to London departing on December 10th and returning on January 5th. Can you provide me with the available options, including airlines, flight times, and prices" 

messages = [
    SystemMessage(content="You're a helpful flight expert"),
    HumanMessage(content=text),
]

chat_model.predict_messages(messages) 


AIMessage(content="Certainly! Here are some available options for direct flights from New York to London departing on December 10th and returning on January 5th:\n\n1. British Airways:\n   - Departure: December 10th, 6:00 PM from JFK Airport\n   - Return: January 5th, 8:30 AM from Heathrow Airport\n   - Price: $850\n\n2. Virgin Atlantic:\n   - Departure: December 10th, 7:30 PM from Newark Airport\n   - Return: January 5th, 9:00 AM from Heathrow Airport\n   - Price: $900\n\n3. American Airlines:\n   - Departure: December 10th, 8:00 PM from JFK Airport\n   - Return: January 5th, 10:30 AM from Heathrow Airport\n   - Price: $950\n\n4. Delta Air Lines:\n   - Departure: December 10th, 9:30 PM from JFK Airport\n   - Return: January 5th, 11:00 AM from Heathrow Airport\n   - Price: $920\n\nPlease note that prices are subject to change and availability may vary. It's always recommended to check with the airlines or a trusted travel agency for the most up-to-date information and to book your pref

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Look at the following conversation {conversation} from the following service area {service_area} on {event_date_time} and returna sentiment"
)

conversation = """Customer: My new bike is missing a wheel!\
Chatbot: I'm sorry to hear that. Could I have your order number, please?\
Customer: It's #54321.\
Chatbot: Thank you. We'll send a replacement wheel today, and it'll arrive in two days.\
Customer: Make sure it does. This has been a hassle.\
Chatbot: Understandably so, and we apologize. You’ll also get a 20% discount on your next order for the inconvenience.\
Customer: Fine, thank you.\
Chatbot: You're welcome, and the confirmation is on its way. If there’s more I can do for you, just let me know."""

reminder_message = prompt_template.format(
    conversation=conversation,
    service_area="complaints",
    event_date_time="2023-10-19 14:30:00"
)
reminder_message



In [ ]:
from langchain.prompts import ChatPromptTemplate 

chat_template = ChatPromptTemplate.from_messages( 

    [ 

("system", "You are a health advisory bot for HealthHub Clinic. You can answer questions from the patient called {name}"), 

("ai", "Hi, {name} please ask me your question."), 

 ("human", "{user_input}"), 

    ] 

) 





messages = chat_template.format_messages(name="Lucy", user_input="What are the symptoms of the flu?")

messages


llm = ChatOpenAI()
llm(messages)



In [ ]:
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, validator

class ProductivityChatPromptTemplate(ChatPromptTemplate, BaseModel):
    @validator("input_variables")
    def validate_input_variables(cls, v):
        required_vars = {"task", "time_available", "user_preferences"}
        if not required_vars.issubset(v):
            raise ValueError(f"Input variables must include: {required_vars}")
        return v

    def format_messages(self, **kwargs) -> str:
        messages = [
            ("system", "You are a virtual productivity assistant."),
            ("human", f"I need to {kwargs['task']} and I only have {kwargs['time_available']}."),
            ("human", f"My preference is to {kwargs['user_preferences']}."),
            ("ai", "Based on your task and preferences, here's my advice:")
            # The AI's response would be generated by the language model following this prompt.
        ]
        return self.construct_chat(messages)


In [ ]:
import inspect


def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

In [ ]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

PROMPT = """\
Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}
Source Code:
{source_code}
Explanation:
"""


class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompt to be sent to the language model
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

    def _prompt_type(self):
        return "function-explainer"
    
    
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)
print(prompt)


In [ ]:
#Filename: Parsing to a JSON dictionary  


from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

intent_schema = ResponseSchema(name="intents",
                             description="Format the output as JSON object consisting of a key: intents, the intents key will be a list of objects with the following keys: intent, utterance, category")
response_schemas = [intent_schema]

intent_output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

intent_format_instructions = intent_output_parser.get_format_instructions()

print(intent_format_instructions)

intent_template = """
Create intents and utterances for a chatbot which will answer questions about a college, \ 
Create 10 examples of intents for the following categories: facilities and course_information \
 ensure that each intent has 10 utterances, create 5 long tail and 5 more common utterances \

{intent_format_instructions}

"""
prompt_template = ChatPromptTemplate.from_template(intent_template)
                                                           
messages = prompt_template.format_messages(intent_format_instructions=intent_format_instructions)
#messages = prompt_template.format_messages(intent_examples=intents)
chat = ChatOpenAI(temperature=0.0, model="gpt-4")
response = chat(messages)

# this will be type str
type(response.content)
                               
output_dict = intent_output_parser.parse(response.content)

output_dict.get('intents')

In [1]:
#Filename: Basic Langchain chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt = ChatPromptTemplate.from_template("""
    Translate this text: {text} to {language} 
"""
)
language = "French"
text = "what is the capital of england"
runnable  = prompt
runnable.invoke({"text":text,"language":language})




ChatPromptValue(messages=[HumanMessage(content='\n    Translate this text: what is the capital of england to French \n')])

In [6]:
#Filename: Sequential chain transcript processor 
import json
from langchain.chat_models import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.schema.runnable import RunnableLambda

# Initialize ChatGPT model
chat_model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

# Load transcripts data
with open('transcripts.json', 'r') as file:
    transcripts = json.load(file)

# Define response schemas
response_schemas = [
    ResponseSchema(name="transactional_transcripts", description="Format the output as JSON list of conversations with the same JSON format as the input,add an category key to each conversation", type="list"),
    ResponseSchema(name="faq", description="Format the output as JSON list of conversations with the same JSON format as the input, add an category key to each conversation ", type="list"),
]

# Define chat transcript template with placeholders for transcripts and format instructions
transcript_template = "Look at the following chat transcripts {transcripts} and categorize them into FAQ and transactional conversations in the following format {format_instructions}"


# Define prompt templates
transactional_categorization_prompt_template = HumanMessagePromptTemplate.from_template(transcript_template)

# Create output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get format instructions
format_instructions = output_parser.get_format_instructions()

# Create prompts
prompt = ChatPromptTemplate(
    messages=[transactional_categorization_prompt_template],
    input_variables=["transcripts"],
    partial_variables={"format_instructions": format_instructions},
)

# Define chain for transactional categorization
chain_one = prompt | chat_model | output_parser

#chain_one_result = chain_one.invoke({"transcripts": transcripts})

# Define intent response schemas
intent_response_schemas = [
    ResponseSchema(name="transactional_intents", description="Format the output as JSON list of conversation transcripts using the same format from the transactional_transcript list, add an intent key to each conversation", type="list"),
]

intent_transcript_template = "Look at the following chat transcripts {transactional_transcripts} Cluster these conversations by intent {intent_format_instructions}"


# Create intent prompt
intent_clustering_prompt_template = HumanMessagePromptTemplate.from_template(intent_transcript_template)

# Create intent output parser
intent_output_parser = StructuredOutputParser.from_response_schemas(intent_response_schemas)

# Get intent format instructions
intent_format_instructions = intent_output_parser.get_format_instructions()

# Create prompt for intent clustering
prompt_two = ChatPromptTemplate(
    messages=[intent_clustering_prompt_template],
    input_variables=["transactional_transcripts"],
    partial_variables={"intent_format_instructions": intent_format_instructions},
)

# Create chain for intent clustering
chain2 = (
    {"transactional_transcripts": chain_one}
    | prompt_two
    | chat_model
    | intent_output_parser
)

# Pass transcripts through chain inputs
chain_two_result = chain2.invoke({"transcripts": transcripts})

In [15]:
print(chain_two_result.get('transactional_intents')[0])

{'live_chat_transcript': [{'role': 'customer', 'message': 'Hello, I need to schedule a maintenance appointment for my ElectricZ Model 3. Can you help with that?'}, {'role': 'support_agent', 'message': "Certainly! Can you please provide your car's VIN and let me know what type of maintenance you require?"}, {'role': 'customer', 'message': 'My VIN is XYZ12345, and I need a routine checkup.'}, {'role': 'support_agent', 'message': "Thank you! I've scheduled your maintenance appointment. You'll receive a confirmation email shortly. Is there anything else I can assist you with?"}], 'category': 'transactional', 'intent': 'Schedule Maintenance'}


In [57]:
#filename: Parallel Chains in Langchain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel
from langchain.schema import StrOutputParser

model = ChatOpenAI(model="gpt-4-1106-preview")

# prompt for querying news articles about a topic
news_prompt = ChatPromptTemplate.from_template("summarize recent news articles about {topic}")

# Chain for querying scientific papers about a topic
academic_prompt = ChatPromptTemplate.from_template("summarize recent scientific papers about {topic}")

# Chain for querying general web information about a topic
web_info_prompt = ChatPromptTemplate.from_template("provide a general overview of {topic} from web sources")


# Create a RunnableParallel instance with the three chains
# parallel_chain = RunnableParallel(news=news_chain, academic=academic_chain, web_info=web_info_chain)

parallel = RunnableParallel(
    news = news_prompt | model,
    academic = academic_prompt | model,
    web_info = web_info_prompt | model
)

summarise_prompt = ChatPromptTemplate.from_template("""
summarize the following information from these different sources:

News source: {news}
Academic: {academic}
Web: {web_info}

Summary:
""")



# Invoke the parallel chain with a specific topic
#results = parallel_chain.invoke({"topic": "artificial intelligence"})

summarise_chain = parallel | summarise_prompt | model

summarise_output = summarise_chain.invoke({"topic": "artificial intelligence"})

print(summarise_output)


content="Recent trends in artificial intelligence (AI) highlight its expanding role in various sectors and the associated ethical considerations. In healthcare, AI is improving patient outcomes through predictive analytics, diagnostic assistance, and treatment personalization, with capabilities matching those of human experts in some cases. Ethical AI developments are gaining attention, with a focus on creating transparent, unbiased systems that respect privacy, alongside calls for stricter regulations. AI's involvement in creative industries raises questions about its impact on creativity, while its role in automation sparks debates on job displacement and creation.\n\nAdvancements in natural language processing (NLP) are notable, with AI models like GPT-3 generating human-like text for applications such as customer service and coding. Autonomous vehicles continue to progress, facing technological and regulatory challenges. Robotic process automation (RPA) is streamlining business ope

In [56]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate  
from langchain.schema.runnable import RunnableParallel, RunnableSequence

model = ChatOpenAI()

joke_prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
poem_prompt = ChatPromptTemplate.from_template("write a 2-line poem about {topic}")

parallel = RunnableParallel(
    joke = joke_prompt | model,
    poem = poem_prompt | model
)

summary_prompt = ChatPromptTemplate.from_template("""
Summarize the joke and poem about

Joke: {joke}
Poem: {poem}

Summary:
""")


sequence = parallel | summary_prompt | model

output = sequence.invoke({"topic": "bears"})
print(output)

content='The joke is about bears never getting caught gambling because they always play with a "bear" minimum. The poem describes bears as wild and mighty guardians of nature that roam freely in deep forests.'


In [13]:
#filename: Routing Chains

from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain.prompts import PromptTemplate
from typing import Literal
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain.pydantic_v1 import BaseModel
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

# Initialize the language model
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

# Maintenance Department Template
maintenance_template = """You are an assistant at a car service center. \
You help customers book their cars for service. You will collect details such as the customer's name, \
car registration number, and their preferred date and time for the car collection.

Here is the service booking request:
{input}"""
maintenance_prompt = PromptTemplate.from_template(maintenance_template)

# Car Information Department Template
car_info_template = """You are knowledgeable about various car models and their features. \
You can provide detailed information about car specifications, models, and performance. \
If a question is outside your expertise, you recommend contacting the car manufacturer.

Here is the question:
{input}"""
car_info_prompt = PromptTemplate.from_template(car_info_template)

# Accounts Department Template
accounts_template = """You are well-versed in account management for car subscriptions. \
You can answer questions about billing, payment methods, and subscription plans. \
In cases of specific account issues, you advise contacting the accounts department directly.

Here is the question:
{input}"""
accounts_prompt = PromptTemplate.from_template(accounts_template)

# General Prompt Template for Other Queries
general_prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the FAQ question as accurately as you can.\n\n{input}"
)

# Branching Logic Based on Department
prompt_branch = RunnableBranch(
    (lambda x: x["topic"] == "maintenance", maintenance_prompt),
    (lambda x: x["topic"] == "car_info", car_info_prompt),
    (lambda x: x["topic"] == "accounts", accounts_prompt),
    general_prompt,
)


# Topic Classifier for Department Selection
class TopicClassifier(BaseModel):
    "Classify the topic of the user question"
    topic: Literal["maintenance", "car_info", "accounts", "general"]
    "The topic of the user question. One of 'maintenance', 'car_info', 'accounts', or 'general'."

classifier_function = convert_pydantic_to_openai_function(TopicClassifier)
llm = ChatOpenAI().bind(
    functions=[classifier_function], function_call={"name": "TopicClassifier"}
)
parser = PydanticAttrOutputFunctionsParser(
    pydantic_schema=TopicClassifier, attr_name="topic"
)
classifier_chain = llm | parser

    
# Final Chain Assembly
final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
    | prompt_branch
    | ChatOpenAI()
    | StrOutputParser()
)

# Example Invocation
final_chain.invoke(
    #{"input": "How do I update my payment method for my car subscription?"}
    #{"input": "which is car with the longest range battery?"}
    {"input": "whats the benefit of a subscription car"}
)


"A subscription car, also known as a car subscription service or car-as-a-service, is a relatively new concept in the automotive industry. It offers an alternative to traditional car ownership or leasing by providing individuals with a flexible and convenient way to access a vehicle.\n\nThe benefits of a subscription car can vary depending on the specific service provider, but here are some common advantages:\n\n1. Flexibility: Car subscriptions typically offer more flexibility compared to traditional ownership or leasing options. Subscribers can often choose the duration of their subscription, ranging from a few weeks to several months, and may have the option to switch to different vehicles within the service's fleet.\n\n2. Convenience: Car subscriptions often include various services in a single monthly payment, such as vehicle registration, insurance, maintenance, and roadside assistance. This eliminates the need for separate arrangements and simplifies the overall ownership experi